In [1]:
from icecube import icetray, dataio, dataclasses, simclasses, clsim
from icecube.icetray import I3Units, OMKey, I3Frame
from icecube.dataclasses import ModuleKey
from os.path import expandvars
import numpy as np
#import matplotlib.pylab as plt
from scipy import stats
from scipy.optimize import minimize
from scipy.stats.distributions import chi2
from likelihoodHelpers import log_likelihood_biGauss, log_likelihood_doublePeak, likelihood_ratio_doublePeak, likelihood_ratio_biGauss, biGauss, double_peak
import scipy, csv
import matplotlib.pyplot as plt

In [3]:
file_number = 713
f = open('/data/p-one/akatil/analysis/RecoPulses/RecoPulses_from_file_'+str(file_number)+'_time_charge.csv', 'w')

with f:
    fnames = ['file', 'frame', 'lepton', 'DOM', 'string', 'recoPulse_time', 'recoPulse_charge']
    writer = csv.writer(f)
    writer.writerow(fnames)
    for i in range(file_number, file_number+1):
        file_name = dataio.I3File('/data/p-one/akatil/step_5_medium_water/NuTau_NuE_20Events/step_5_'+str(i)+'_medium_water_custom_mDOM_recoPulse.i3.gz')
        frame_num = 0
        for frame in file_name:
            mctree = frame["I3MCTree"]
            primary = mctree.primaries
            lepton = dataclasses.I3MCTree.first_child(mctree, primary[0].id)

            recoPulseMap = frame['I3RecoPulses']
        
            for omkey in recoPulseMap.keys():
                recoPulseList = recoPulseMap[omkey]
                for recoPulse in recoPulseList:
                    recoPulse_timeList = recoPulse.time
                    recoPulse_chargeList = recoPulse.charge
                
                    info_list = [i, frame_num, lepton.type, omkey[1], omkey[0], recoPulse_timeList, recoPulse_chargeList]
                
                    writer.writerow(info_list)
                    
            frame_num += 1


In [1]:
#with f:
fnames = ['file', 'frame', 'lepton_type', 'DOM', 'string', 'recoPulses_time', 'recoPulses_charge']
#writer = csv.writer(f)
#writer.writeheader()
#writer.writerow(fnames)
for i in range(file_number, file_number+1):
    file_name = dataio.I3File('/data/p-one/akatil/step_5_medium_water/NuTau_NuE_20Events/step_5_'+str(i)+'_medium_water_custom_mDOM_recoPulse.i3.gz')
    frame_num = 0
    for frame in file_name:
        mctree = frame["I3MCTree"]
        primary = mctree.primaries
        lepton = dataclasses.I3MCTree.first_child(mctree, primary[0].id)

        recoPulseMap = frame['I3RecoPulses']
        
        for omkey in recoPulseMap.keys():
            recoPulseList = recoPulseMap[omkey]
            #recoPulse_timeList = [recoPulse.time for recoPulse in recoPulseList]
            #recoPulse_chargeList = [recoPulse.charge for recoPulse in recoPulseList]
                
            recoPulse_timeList = np.array([recoPulse.time for recoPulse in recoPulseList])
            recoPulse_chargeList = np.array([recoPulse.charge for recoPulse in recoPulseList])
                
            info_list = [i, frame_num, lepton.type, omkey[1], omkey[0], recoPulse_timeList, recoPulse_chargeList]
                
                #writer.writerow({'file' : info_list[0], 'frame' : info_list[1], 'lepton_type' : info_list[2],
                 #                   'DOM' : info_list[3], 'string' : info_list[4], 'recoPulses_time' : info_list[5],
                  #                  'recoPulses_charge' : info_list[6]})  
                
                #writer.writerow(info_list)
                
                #recoPulse_timeList = np.array([recoPulse.time for recoPulse in recoPulseList])
                #recoPulse_chargeList = np.array([recoPulse.charge for recoPulse in recoPulseList])
            
            '''
            Removing DOMs with hits less than 150 Hits
            '''
            if sum(recoPulse_chargeList) < 100:
                continue

            '''
            Calculating the mean and removing the tails
            '''

            #mean = recoPulse_timeList.mean()
            mean = sum(recoPulse_timeList*recoPulse_chargeList)/sum(recoPulse_chargeList) #mean is weighted
            select_time = recoPulse_timeList[(recoPulse_timeList > mean-50) & (recoPulse_timeList < mean+50)]
            select_charge = recoPulse_chargeList[(recoPulse_timeList > mean-50) & (recoPulse_timeList < mean+50)]
            #print('SELECT CHARGE', select_charge, select_time, mean, recoPulse_timeList, recoPulse_chargeList)

            if len(select_time) < 10:
                continue

            mean_select_time = sum(select_time*select_charge)/sum(select_charge)
            max_hitTimes = recoPulse_timeList[(recoPulse_timeList > (mean_select_time-100))&(recoPulse_timeList < (mean_select_time+100))]
            max_charge = recoPulse_chargeList[(recoPulse_timeList > (mean_select_time-100))&(recoPulse_timeList < (mean_select_time+100))]


            #[using zscore to remove the effect of outliers from the analysis]
            z = stats.zscore(max_hitTimes)
            max_hitTimes = max_hitTimes[(z>-4.0)&(z < 0.1)]
            max_charge = max_charge[(z>-4.0)&(z < 0.1)]

            if len(max_hitTimes) < 10:
                continue

            #Shifting mean to zero
            max_hitTimes_mean = sum(max_hitTimes*max_charge)/sum(max_charge)
            timestamps = max_hitTimes - max_hitTimes_mean
            final_mean = timestamps.mean()

            '''
            Histogramming the data from simulation
            '''
            print('Now Histogramming')
            bins = np.arange(min(timestamps), max(timestamps), 3)
            num, bin_edges = np.histogram(timestamps, bins=bins, weights=max_charge)
            bin_centers = (bin_edges[:-1]+bin_edges[1:])/2

            #removing bins with < 0 entries ---> Recommended for binned likelihood
            entries_in_bins = num#[num > 0]
            bin_centers = bin_centers#[num > 0]

            #Getting data for the chi2 fit
            #chi2_entries_in_bins = entries_in_bins[entries_in_bins > 10]
            #chi2_bin_centers = bin_centers[entries_in_bins > 10]

            #num_dataPoints = len(chi2_entries_in_bins)
            #area_data = sum(entries_in_bins)
            #mean_entries = entries_in_bins.mean()

            '''
            Fitting bifurcated Gaussian and double bifurcated gaussian to
            the mcpe hit time distributions for both tau and electron.
            '''

            #Single Peak

            nll = lambda *args: log_likelihood_biGauss(*args)
            initial_biGauss = np.array([final_mean, 50, 5, max(entries_in_bins)])
            bnds_biGauss = ((min(bin_centers), max(bin_centers)), (0, 500), (0, 10), (0, 1e4))
            soln_biGauss = minimize(log_likelihood_biGauss, initial_biGauss,
                                        args=(entries_in_bins, bin_centers),
                                        method='TNC',
                                        bounds = bnds_biGauss)

            #Double Peak

            nll = lambda *args: log_likelihood_doublePeak(*args)
            initial_doublePeak = np.array([min(bin_centers)+10, 20, 1, max(entries_in_bins), final_mean, 20, 1, max(entries_in_bins)])
            bnds_doublePeak = ((min(bin_centers), final_mean), (0, 500), (0, 10), (0, 1e4),
                                    (final_mean, max(bin_centers)), (0, 500), (0, 10), (0,1e4))
            soln_doublePeak = minimize(log_likelihood_doublePeak, initial_doublePeak,
                                        args=(entries_in_bins, bin_centers),
                                        method='TNC',
                                        bounds=bnds_doublePeak)

            '''
            (x, y) values for the fit
            '''
            #x = bin_centers
            x = np.linspace(min(bin_centers), max(bin_centers), 1000)
            y_biGauss = biGauss(x, soln_biGauss.x[0],
                                    soln_biGauss.x[1], soln_biGauss.x[2], soln_biGauss.x[3])
            y_doublePeak = double_peak(x, soln_doublePeak.x[0], soln_doublePeak.x[1],
                                            soln_doublePeak.x[2], soln_doublePeak.x[3], soln_doublePeak.x[4],
                                            soln_doublePeak.x[5], soln_doublePeak.x[6], soln_doublePeak.x[7])



            plt.figure(figsize=(10,9))
            _ = plt.hist(timestamps, bins=bins, weights=max_charge, histtype='step')
            plt.plot(x, y_biGauss, '-', c = 'k')
            plt.plot(x, y_doublePeak, '-', c = 'r')
            plt.title(str(lepton.type))
                
        frame_num += 1
            

NameError: name 'file_number' is not defined

In [1]:
from icecube import icetray, dataio, dataclasses, simclasses, clsim
from icecube.icetray import I3Units, OMKey, I3Frame
from icecube.dataclasses import ModuleKey
from os.path import expandvars
import numpy as np
#import matplotlib.pylab as plt
from scipy import stats
from scipy.optimize import minimize
from scipy.stats.distributions import chi2
from likelihoodHelpers import log_likelihood_biGauss, log_likelihood_doublePeak, likelihood_ratio_doublePeak, likelihood_ratio_biGauss, biGauss, double_peak
import scipy, csv